# Creating Crosswalk with DLA Locode, NTD ID and ITP ID

In [1]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [3]:
from calitp.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/'

pd.set_option('display.max_columns', None)

In [5]:
dla = pd.read_excel(f"{GCS_FILE_PATH}locodes_updated7122021.xlsx")

In [14]:
dla = dla>>select(_['Agency Locode'], _['Agency Name'])
dla = dla.rename(columns = {'Agency Name':'Name'})

In [10]:
itp = pd.read_csv(f"{GCS_FILE_PATH}organizations_All_Organizations.csv")

In [12]:
itp = itp>>select(_.Name, _['NTP ID'], _['ITP ID'])


In [17]:
ids = pd.merge(itp, dla, on='Name', how="outer", indicator=True)

In [20]:
ids._merge.value_counts()

right_only    924
left_only     711
both          117
Name: _merge, dtype: int64

In [23]:
(ids>>filter(_._merge=='both'))

,Name,NTP ID,ITP ID,Agency Locode,_merge
4,Access Services,90157,1.0,6312.0,both
13,Alpine County,9R02-91116,9.0,5931.0,both
15,Amador County,NaN,NaN,5926.0,both
16,Amador County Transportation Commission,NaN,NaN,6127.0,both
28,Antelope Valley Transit Authority,90121,16.0,6166.0,both
...,...,...,...,...,...
798,Ventura County Transportation Commission,90164,380.0,6155.0,both
803,Victor Valley Transit Authority,90148,360.0,6261.0,both
815,Western Contra Costa Transit Authority,90159,368.0,6226.0,both
822,Yolo County Transportation District,90090,372.0,6195.0,both


In [24]:
(ids>>filter(_._merge=='both')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 4 to 824
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Name           117 non-null    object  
 1   NTP ID         56 non-null     object  
 2   ITP ID         66 non-null     float64 
 3   Agency Locode  117 non-null    float64 
 4   _merge         117 non-null    category
dtypes: category(1), float64(2), object(2)
memory usage: 4.8+ KB


In [27]:
nomatch = ids>>filter(_._merge!='both')

In [30]:
nomatch= to_snakecase(nomatch)

In [31]:
nomatch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1635 entries, 0 to 1751
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   name           1635 non-null   object  
 1   ntp_id         143 non-null    object  
 2   itp_id         308 non-null    float64 
 3   agency_locode  924 non-null    float64 
 4   _merge         1635 non-null   category
dtypes: category(1), float64(2), object(2)
memory usage: 65.6+ KB


In [38]:
(nomatch.query('itp_id.notnull() and ntp_id.notnull()')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 10 to 826
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   name           135 non-null    object  
 1   ntp_id         135 non-null    object  
 2   itp_id         135 non-null    float64 
 3   agency_locode  0 non-null      float64 
 4   _merge         135 non-null    category
dtypes: category(1), float64(2), object(2)
memory usage: 5.5+ KB
